## Merton Model Testing - Ryan ##

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize, root, differential_evolution
import matplotlib.pyplot as plt


In [2]:
def compute_merton_pd_new(E, sigma_E, D, r, T=1.0, debug = False, plot_mesh = False):
    # Bad input
    if E <= 0 or r < 0:
        return np.nan
    
    # Shortcut: if leverage is very low and sigma_E is not high, assume PD ≈ 0 to avoid solver convergence issues
    if (D == 0) or (sigma_E == 0) or ((D / E < 0.2) and (sigma_E < 0.6)):
        return 0.0
    
    # Clip to aid the solver
    sigma_E = np.clip(sigma_E, 0.05, 2.0)
    
    # Sqared residuals from first two equations
    def equations(vars):
        V, sigma_V = vars
        if V <= 0 or sigma_V <= 0:
            #print('First Exception')
            return [1e10, 10]  # large penalty for invalid values
        try:
            d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
            d2 = d1 - sigma_V * np.sqrt(T)
            eq1 = V * norm.cdf(d1) - D * np.exp(-r * T) * norm.cdf(d2) - E
            eq2 = (V / E) * norm.cdf(d1) * sigma_V - sigma_E
            #print(V, sigma_V)
            return [eq1, eq2]
        except:
            #print('Last Exception')
            return [1e10, 10]
        
    """V0 = max(E + D, 1e6)
    sigma_V0 = np.clip(sigma_E * 0.9, 0.05, 2.0)
    bounds = [(1e6, 1e14), (0.01, 2.0)]
    result = differential_evolution(equations, bounds = bounds, maxiter=1000)
    if debug == True:
        print(result)"""
    
    
    V_range = np.linspace(E + 0.1 * D, E + 5 * D, 100)
    sigma_V_range = np.linspace(0.05, 1.5, 100)
    residual_grid = np.zeros((len(sigma_V_range), len(V_range)))

    best_sol = None
    best_residual = np.inf

    for i, sigma_V0 in enumerate(sigma_V_range):
        for j, V0 in enumerate(V_range):
            result = root(equations, x0=[V0, sigma_V0], method='lm')
            if result.success:
                residual = np.linalg.norm(result.fun)
                residual_grid[i, j] = residual
                if residual < best_residual:
                    best_residual = residual
                    best_sol = result.x
                    if debug:
                        print(f"New best: residual={residual:.4e}, V0={V0:.2e}, sigma_V0={sigma_V0:.2f}")
            else:
                residual_grid[i, j] = np.nan
    if debug == True:
        print(best_sol)
    if plot_mesh:
        V_mesh, sigma_V_mesh = np.meshgrid(V_range, sigma_V_range)
        fig = plt.figure(figsize=(10, 6))
        cp = plt.contourf(V_mesh, sigma_V_mesh, residual_grid, levels=50, cmap='viridis')
        plt.colorbar(cp, label='Residual')
        plt.xlabel("V initial guess")
        plt.ylabel("sigma_V initial guess")
        plt.title("Residuals Heatmap for Merton Model Calibration")
        plt.tight_layout()
        plt.show()

    if best_sol is not None:
        V_opt, sigma_V_opt = best_sol
        d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))
        return norm.cdf(-d2)
    else:
        return np.nan


"""
    if result.success:
        V_opt, sigma_V_opt = result.x
        d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))
        pd = norm.cdf(-d2)
        return pd
    else:
        return np.nan"""

'\n    if result.success:\n        V_opt, sigma_V_opt = result.x\n        d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))\n        pd = norm.cdf(-d2)\n        return pd\n    else:\n        return np.nan'

In [3]:
def compute_merton_pd_old(E, sigma_E, D, r, T=1.0):
    # Bad input
    if E <= 0 or r < 0:
        return np.nan
    
    # Shortcut: if leverage is very low and sigma_E is not high, assume PD ≈ 0 to avoid solver convergence issues
    if (D == 0) or (sigma_E == 0) or ((D / E < 0.2) and (sigma_E < 0.6)):
        return 0.0
    
    # Clip to aid the solver
    sigma_E = np.clip(sigma_E, 0.05, 2.0)
    
    # Sqared residuals from first two equations
    def equations(vars):
        V, sigma_V = vars
        if V <= 0 or sigma_V <= 0:
            return 1e10  # large penalty for invalid values
        try:
            d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
            d2 = d1 - sigma_V * np.sqrt(T)
            eq1 = V * norm.cdf(d1) - D * np.exp(-r * T) * norm.cdf(d2) - E
            eq2 = (V / E) * norm.cdf(d1) * sigma_V - sigma_E
            print(V, sigma_V)
            return eq1**2 + eq2**2
        except:
            return 1e10

    V0 = max(E + D, 1e6)
    sigma_V0 = np.clip(sigma_E * 0.9, 0.05, 2.0)
    bounds = [(1e6, 1e14), (0.01, 2.0)]
    result = minimize(equations, x0=[V0, sigma_V0], bounds=bounds, method='L-BFGS-B', options={'ftol': 1e-3})
    print(f"V, sigma_V: {result.x}")
    print(f"Residuals: {result.fun}")
    if result.success:
        V_opt, sigma_V_opt = result.x
        d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))
        pd = norm.cdf(-d2)
        return pd
    else:
        return np.nan

In [4]:
merton_data = pd.read_csv(r"merton_model_output.csv")

In [5]:
merton_data.head()
len(merton_data)

441050

In [6]:
merton_data_converged = merton_data[merton_data["pd_valid"] == True]
len(merton_data_converged)

438957

In [5]:
merton_data_nonconverge = merton_data[merton_data["pd_valid"] == False]
len(merton_data_nonconverge)
merton_data_nonconverge.head()

,date,permno,tic,conm,PRC,atq,dlcq,dlttq,SHROUT,market_cap,total_debt,leverage,log_return,equity_volatility,rf,merton_pd,pd_valid
2121,2014-06-09,14593,AAPL,APPLE INC,93.70,2.059890e+11,0.0,1.696200e+10,6029667.0,5.649798e+11,8.481000e+09,0.082344,-1.930035,1.946300,2.62,NaN,False
2123,2014-06-11,14593,AAPL,APPLE INC,93.86,2.059890e+11,0.0,1.696200e+10,6029667.0,5.659445e+11,8.481000e+09,0.082344,-0.004147,1.946335,2.65,NaN,False
2124,2014-06-12,14593,AAPL,APPLE INC,92.29,2.059890e+11,0.0,1.696200e+10,6029667.0,5.564780e+11,8.481000e+09,0.082344,-0.016869,1.946355,2.58,NaN,False
2125,2014-06-13,14593,AAPL,APPLE INC,91.28,2.059890e+11,0.0,1.696200e+10,6029667.0,5.503880e+11,8.481000e+09,0.082344,-0.011004,1.946304,2.60,NaN,False
2126,2014-06-16,14593,AAPL,APPLE INC,92.20,2.059890e+11,0.0,1.696200e+10,6029667.0,5.559353e+11,8.481000e+09,0.082344,0.010028,1.946357,2.61,NaN,False


In [6]:
merton_data_converged.describe()

,permno,PRC,atq,dlcq,dlttq,SHROUT,market_cap,total_debt,leverage,log_return,equity_volatility,rf,merton_pd
count,362879.000000,362879.000000,3.628790e+05,3.628790e+05,3.628790e+05,3.628790e+05,3.628790e+05,3.628790e+05,362879.000000,362879.000000,362879.000000,362879.000000,362879.000000
mean,48726.221578,2455.433667,2.325942e+11,2.579563e+10,3.320529e+10,1.780045e+06,1.390025e+11,4.239827e+10,0.256681,-0.002921,2.013645,2.821379,0.000344
std,29164.943187,28428.356218,4.801447e+11,7.829684e+10,6.086585e+10,2.221494e+06,2.405342e+11,1.041810e+11,0.163613,0.897100,13.988776,1.106609,0.013206
min,10104.000000,0.350000,3.646810e+08,0.000000e+00,0.000000e+00,5.520000e+02,8.360609e+08,0.000000e+00,0.000000,-7.346823,0.090416,0.520000,0.000000
25%,18729.000000,44.160000,3.340800e+10,4.000000e+08,5.801000e+09,5.999820e+05,4.941775e+10,3.789000e+09,0.147746,-0.007974,0.193687,1.960000,0.000000
50%,49680.000000,71.000000,6.512100e+10,1.800000e+09,1.310500e+10,1.100286e+06,8.668884e+10,9.211500e+09,0.234005,0.000500,0.245729,2.680000,0.000000
75%,77702.000000,125.890000,1.653570e+11,5.732000e+09,2.991900e+10,1.882048e+06,1.561184e+11,2.275400e+10,0.340765,0.008974,0.328988,3.700000,0.000000
max,92655.000000,724040.000000,4.210048e+12,5.628570e+11,5.116530e+11,2.920640e+07,3.915300e+12,7.260785e+11,1.039717,7.406942,116.571964,5.260000,1.000000


In [7]:
auto_drop_merton = merton_data[
    (merton_data['total_debt'] == 0) | 
    (merton_data['equity_volatility'] == 0) | 
    (((merton_data['total_debt'] / merton_data['market_cap']) < .2) & (merton_data['equity_volatility'] < .6))
]

In [8]:
len(auto_drop_merton)

288585

In [9]:
merton_converge_not_auto = merton_data_converged.drop(auto_drop_merton.index)

In [10]:
merton_converge_not_auto.describe().loc['mean']

permno               5.688536e+04
PRC                  1.063252e+04
atq                  8.248819e+11
dlcq                 1.159169e+11
dlttq                1.025688e+11
SHROUT               2.379644e+06
market_cap           1.177250e+11
total_debt           1.672012e+11
leverage             2.921293e-01
log_return          -1.588132e-02
equity_volatility    8.839949e+00
rf                   2.731226e+00
merton_pd            1.678412e-03
Name: mean, dtype: float64

In [11]:
merton_data_nonconverge.describe().loc['mean']

permno               5.093782e+04
PRC                  4.127842e+03
atq                  3.602884e+11
dlcq                 2.731390e+10
dlttq                5.264697e+10
SHROUT               1.904643e+06
market_cap           1.028473e+11
total_debt           5.363739e+10
leverage             2.933262e-01
log_return           1.476883e-02
equity_volatility    3.216877e+00
rf                   2.847869e+00
merton_pd                     NaN
Name: mean, dtype: float64

In [12]:
len(merton_converge_not_auto) / (len(merton_data_nonconverge) + len(merton_converge_not_auto))

0.48728560653264685

In [13]:
comparison = (pd.DataFrame([merton_converge_not_auto.describe().loc['mean'], merton_data_nonconverge.describe().loc['mean']], index=['Converge', 'Non-Converge'])).transpose()

In [14]:
print(comparison)

                       Converge  Non-Converge
permno             5.688536e+04  5.093782e+04
PRC                1.063252e+04  4.127842e+03
atq                8.248819e+11  3.602884e+11
dlcq               1.159169e+11  2.731390e+10
dlttq              1.025688e+11  5.264697e+10
SHROUT             2.379644e+06  1.904643e+06
market_cap         1.177250e+11  1.028473e+11
total_debt         1.672012e+11  5.363739e+10
leverage           2.921293e-01  2.933262e-01
log_return        -1.588132e-02  1.476883e-02
equity_volatility  8.839949e+00  3.216877e+00
rf                 2.731226e+00  2.847869e+00
merton_pd          1.678412e-03           NaN


In [32]:
merton_data.iloc[2123]

date                     2014-06-11
permno                        14593
tic                            AAPL
conm                      APPLE INC
PRC                           93.86
atq                  205989000000.0
dlcq                            0.0
dlttq                 16962000000.0
SHROUT                    6029667.0
market_cap           565944544620.0
total_debt             8481000000.0
leverage                   0.082344
log_return                -0.004147
equity_volatility          1.946335
rf                             2.65
merton_pd                       NaN
pd_valid                      False
Name: 2123, dtype: object

In [136]:
compute_merton_pd_new(565944544620.0,1.946335,8481000000.0,2.65 / 100, debug = True)

New best: residual=1.2207e-04, V0=5.67e+11, sigma_V0=0.05
New best: residual=2.2204e-16, V0=5.68e+11, sigma_V0=0.05
New best: residual=0.0000e+00, V0=5.68e+11, sigma_V0=0.05
[5.73766556e+11 1.92127576e+00]


0.10625476417896262

In [135]:
compute_merton_pd_old(565944544620.0,1.946335,8481000000.0,2.65 / 100)

574425544620.0 1.7517015
574425553179.6077 1.7517015
574425544620.0 1.75170151
574417685641.0248 0.01
574417694200.5153 0.01
574417685641.0248 0.01000001
573989805559.0231 0.01
573989814112.1377 0.01
573989805559.0231 0.01000001
574203752018.8447 0.01
574203760575.1473 0.01
574203752018.8447 0.01000001
574203745601.6179 1.9658360583067365
574203754157.9205 1.9658360583067365
574203745601.6179 1.9658360683067364
574203750078.9388 0.6012426108863259
574203758635.2415 0.6012426108863259
574203750078.9388 0.601242620886326
574203745601.6024 1.9371295732984761
574203754157.905 1.9371295732984761
574203745601.6024 1.937129583298476
574203748798.0127 0.9834279770698583
574203757354.3153 0.9834279770698583
574203748798.0127 0.9834279870698583
574203749680.8932 0.7200060928798857
574203758237.1958 0.7200060928798857
574203749680.8932 0.7200061028798858
574203749098.1921 0.8938645364452676
574203757654.4948 0.8938645364452676
574203749098.1921 0.8938645464452677
574203749503.4663 0.7729443045917

1.5908188562424526e-08

In [80]:
compute_merton_pd_old(565944544620.0,1.946335,8481000000.0,2.65)

574425544620.0 1.7517015
574425553179.6077 1.7517015
574425544620.0 1.75170151
558657405434.3732 0.01
558657413759.0172 0.01
558657405434.3732 0.01000001
566544128288.0082 0.8811438212558068
566544136730.1736 0.8811438212558068
566544128288.0082 0.8811438312558069
566543338385.7576 0.8811438212558068
566543346827.9111 0.8811438212558068
566543338385.7576 0.8811438312558069
566543739666.879 0.8811438212558068
566543748109.0386 0.8811438212558068
566543739666.879 0.8811438312558069
566543733336.8859 1.9696099224128858
566543741779.0454 1.9696099224128858
566543733336.8859 1.9696099324128857
566543737713.0691 1.2171087707253878
566543746155.2286 1.2171087707253878
566543737713.0691 1.2171087807253878
566543733243.0404 1.9304166334904886
566543741685.2 1.9304166334904886
566543733243.0404 1.9304166434904886
566543736392.9448 1.4277685041521493
566543744835.1044 1.4277685041521493
566543736392.9448 1.4277685141521492
566543737140.6998 1.308444976812733
566543745582.8594 1.308444976812733
56

5.081768136852948e-06

In [89]:
merton_data_nonconverge['merton_pd'] = merton_data_nonconverge.apply(
    lambda row: compute_merton_pd_new(
        E=row['market_cap'],
        sigma_E=row['equity_volatility'],
        D=row['total_debt'],
        r=row['rf'] / 100  # Convert percentage to decimal
    ),
    axis=1
)

merton_data_nonconverge['pd_valid'] = merton_data_nonconverge['merton_pd'].notna()
len(merton_data_nonconverge[merton_data_nonconverge['pd_valid'] == False])

C:\Users\rycba\AppData\Local\Temp\ipykernel_33104\154802804.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merton_data_nonconverge['merton_pd'] = merton_data_nonconverge.apply(
C:\Users\rycba\AppData\Local\Temp\ipykernel_33104\154802804.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merton_data_nonconverge['pd_valid'] = merton_data_nonconverge['merton_pd'].notna()


76

In [92]:
merton_data_nonconverge['merton_pd']

2121      0.106420
2123      0.106255
2124      0.107805
2125      0.108746
2126      0.107865
            ...   
440062    0.000189
440063    0.000214
440065    0.000190
440077    0.000118
440078    0.000104
Name: merton_pd, Length: 78171, dtype: float64

In [94]:
gs = merton_data[merton_data['tic'] == 'GS']

In [100]:
merton_data_nonconverge.loc[180373]

date                              2007-09-24
permno                                 86868
tic                                       GS
conm                 GOLDMAN SACHS GROUP INC
PRC                                210.42999
atq                          1045778000000.0
dlcq                          459520000000.0
dlttq                         188980000000.0
SHROUT                              405655.0
market_cap                85361977593.449997
total_debt                    554010000000.0
leverage                            0.620112
log_return                          0.002141
equity_volatility                   0.276808
rf                                      4.63
merton_pd                                1.0
pd_valid                                True
Name: 180373, dtype: object

In [128]:
compute_merton_pd_new(merton_data_nonconverge.loc[180373]['market_cap'], 
                      merton_data_nonconverge.loc[180373]['equity_volatility'], 
                      merton_data_nonconverge.loc[180373]['total_debt'], 
                      merton_data_nonconverge.loc[180373]['rf'] / 100, debug = True
)

New best: residual=8.5362e+10, V0=1.41e+11, sigma_V0=0.05
New best: residual=8.5362e+10, V0=3.60e+11, sigma_V0=0.05
New best: residual=8.0129e+10, V0=3.88e+11, sigma_V0=0.05
New best: residual=9.4544e-01, V0=4.15e+11, sigma_V0=0.05
New best: residual=5.8984e-01, V0=4.42e+11, sigma_V0=0.05
New best: residual=1.0681e-04, V0=5.79e+11, sigma_V0=0.05
New best: residual=1.5259e-05, V0=6.62e+11, sigma_V0=0.05
[6.14305814e+11 3.84661844e-02]


5.4381840411249855e-05